# Лабораторная работа №6
### "ИНФОРМАЦИОННОЕ ПРОТИВОБОРСТВО"

**Вариант:** 1

**Цель работы:** изучить теоретико-игровую модель информационного противоборства в социальных сетях. Промоделировать информационное управление в рамках игры и определить итоговое мнение агентов.

In [145]:
import logging

import numpy as np

np.set_printoptions(precision=4, suppress=True)
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [146]:
# Размерность матрицы (количество агентов)
N = 10

### 1. Генерация стохастической матрицы
В модели социальной сети рассматриваются агенты ($i \in N = {1,2,..., n}, n \in \mathbb{N}$), составляющие эту сеть, и их
мнения $х_i$, зависящие от дискретного момента времени: $x_i = x_i(t), t = 0, 1, 2, ...$. 

В каждый момент времени мнения агентов изменяются под влиянием мнений других агентов. Степень этого влияния определяется неотрицательной стохастической по строкам матрицей $А = (a_{ij})$, где $a_{ij}$ — степень доверия $і$-го агента $j$-му агенту (в том числе и самому себе).

In [147]:
def generate_trust_matrix(n_agents):
    matrix = np.random.rand(n_agents, n_agents)
    # Нормализуем строки и столбцы, чтобы сумма значений в каждой строке была равна 1.
    return matrix / matrix.sum(axis=1, keepdims=True)


# Функция для проверки стохастичности матрицы
def is_stochastic(matrix):
    rows_sum = np.sum(matrix, axis=1)
    logging.info(f"Сумма по строкам: {rows_sum}")
    # Проверяем, что сумма значений в строках равна 1.
    return np.allclose(rows_sum, 1)


# Генерируем стохастическую матрицу доверия
trust_matrix = generate_trust_matrix(N)
trust_matrix

array([[0.0596, 0.2094, 0.1181, 0.046 , 0.0328, 0.016 , 0.0385, 0.1899,
        0.18  , 0.1098],
       [0.0032, 0.109 , 0.2126, 0.2052, 0.0927, 0.0323, 0.0477, 0.131 ,
        0.1413, 0.0249],
       [0.0682, 0.0126, 0.1556, 0.0454, 0.1117, 0.1621, 0.1633, 0.1262,
        0.0412, 0.1137],
       [0.1085, 0.1336, 0.0953, 0.0602, 0.1047, 0.1242, 0.105 , 0.0594,
        0.1243, 0.0849],
       [0.1633, 0.0894, 0.1479, 0.1555, 0.0803, 0.1615, 0.0187, 0.1337,
        0.024 , 0.0256],
       [0.1716, 0.0772, 0.1494, 0.1157, 0.1022, 0.0814, 0.1034, 0.088 ,
        0.0594, 0.0515],
       [0.2128, 0.0158, 0.1367, 0.1927, 0.0395, 0.0106, 0.0838, 0.0727,
        0.073 , 0.1622],
       [0.118 , 0.0033, 0.1733, 0.0807, 0.0736, 0.1446, 0.1676, 0.0688,
        0.0968, 0.0734],
       [0.1025, 0.1518, 0.0704, 0.1241, 0.042 , 0.1823, 0.0186, 0.0781,
        0.0343, 0.1961],
       [0.1524, 0.1751, 0.1337, 0.1684, 0.004 , 0.0591, 0.1133, 0.0493,
        0.1323, 0.0122]])

In [148]:
is_stochastic(trust_matrix)

Сумма по строкам: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


True

### 2. Мнение агентов

Вектор мнений $X = (x_1, x_2, ..., x_n)$ задан для начального момента времени ($X(0)$) и изменяется следующим образом:
$$x_i(t) = \sum_{j = 0}^{n} a_{ij}x_{j}(t - 1)$$

Этот закон можно записать иначе:
$$X(t) = A \cdot X(t ー 1)$$

При достаточно долгом взаимодействии агентов вектор мнений сходится к итоговому значению:
$$X = A^{\infty} \cdot X(0)$$
, где $A^{\infty} = \lim_{t \to \infty} A^{t}$.

In [149]:
def compute_final_opinions(trust_matrix, initial_opinions, epsilon=1e-6):
    i = 0
    logging.info(f"x({i}) = {initial_opinions}")
    current_opinions = initial_opinions.copy()
    previous_opinions = np.zeros_like(initial_opinions)
    res_trust_matrix = trust_matrix.copy()
    while np.linalg.norm(current_opinions - previous_opinions) > epsilon:
        previous_opinions = current_opinions
        current_opinions = np.dot(trust_matrix, current_opinions)
        res_trust_matrix = np.dot(res_trust_matrix, trust_matrix)
        i += 1
        logging.info(f"x({i}) = {current_opinions}")

    return current_opinions, res_trust_matrix

In [150]:
a, b = 1, 20
# Генерируем начальные мнения агентов.
initial_opinions = np.random.randint(a, b + 1, N)
final_opinions, res_trust_matrix = compute_final_opinions(trust_matrix, initial_opinions)
print(f"После взаимодействия агентов, вектор мнений сходится к значению:\nX = {final_opinions}")
print(f"Результирующая матрица доверия:\n{res_trust_matrix}")

x(0) = [ 6  2  4  6 19  5 20 19  3 18]
x(1) = [ 9.122   8.5571 12.0944  9.3996  8.4644  9.6723 10.0128 10.2833  9.1409
  7.0025]
x(2) = [9.3802 9.8509 9.6576 9.3308 9.7085 9.6184 9.3695 9.7644 9.0447 9.632 ]
x(3) = [9.5636 9.5271 9.5639 9.5319 9.57   9.531  9.4749 9.4951 9.5899 9.4832]
x(4) = [9.5337 9.5412 9.526  9.5365 9.5384 9.5374 9.5333 9.5338 9.5266 9.5391]
x(5) = [9.534  9.5331 9.5341 9.5346 9.5346 9.5341 9.5339 9.5332 9.5363 9.5338]
x(6) = [9.5341 9.5343 9.5341 9.5342 9.534  9.5341 9.5342 9.5343 9.534  9.5342]
x(7) = [9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5341 9.5342 9.5342]
x(8) = [9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342]
x(9) = [9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342]
x(10) = [9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342]


После взаимодействия агентов, вектор мнений сходится к значению:
X = [9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342 9.5342]
Результирующая матрица доверия:
[[0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]]


### 3. Информационное управление
Для информационного управления необходимо наличие нескольких игроков (в данном случае - двух), которые могут влиять на начальные мнения некоторых агентов. Случайным образом выберем число и номера агентов первого игрока и аналогично число и номера агентов второго игрока (они не должны пересекаться, так как эта ситуация выходит за рамки данной работы). Следует обратить внимание, что число агентов влияния первого игрока не обязательно должно совпадать с числом агентов влияния второго игрока.

In [151]:
def get_player_agents_indices(n_agents):
    # Генерируем случайное число агентов для каждого игрока (необязательно равное).
    player1_count = np.random.randint(1, n_agents)
    # Учитываем уже выбранных агентов для первого игрока.
    player2_count = np.random.randint(1, n_agents - player1_count + 1)  

    # Генерируем случайные неповторяющиеся индексы агентов для каждого игрока
    player1_indices = np.random.choice(
        range(n_agents), 
        size=player1_count, replace=False)
    player2_indices = np.random.choice(
        [idx for idx in range(n_agents) if idx not in player1_indices], 
        size=player2_count, replace=False)
    
    player1_indices.sort()
    player2_indices.sort()
    return player1_indices, player2_indices

In [152]:
player1_influence_indices, player2_influence_indices = get_player_agents_indices(N)
print(f"Индексы агентов влияния для первого игрока: {player1_influence_indices}")
print(f"Индексы агентов влияния для второго игрока: {player2_influence_indices}")

Индексы агентов влияния для первого игрока: [0 1 2 3 4 5 6 7 9]
Индексы агентов влияния для второго игрока: [8]


Затем сгенерируем случайным образом управление для агентов влияния первого и второго игроков.
Сформируем вектор начальных мнений агентов с учетом информационного влияния, нейтральным агентам назначим случайные стартовые мнения. 

In [153]:
ABS_MAX_INFLUENCE = 100
player1_influence = np.random.randint(0, ABS_MAX_INFLUENCE)
print(f"Сформированное начальное мнение агентов первого игрока: {player1_influence}")
player2_influence = np.random.randint(-ABS_MAX_INFLUENCE, 0)
print(f"Сформированное начальное мнение агентов второго игрока: {player2_influence}")

Сформированное начальное мнение агентов первого игрока: 1
Сформированное начальное мнение агентов второго игрока: -37


In [154]:
# Модифицируем начальные мнения агентов с учётом влияния.
for i in player1_influence_indices:
    initial_opinions[i] = player1_influence
    
for i in player2_influence_indices:
    initial_opinions[i] = player2_influence

final_opinions, res_trust_matrix = compute_final_opinions(trust_matrix, initial_opinions)
print(f"После взаимодействия агентов, вектор мнений сходится к значению:\nX = {final_opinions}")
print(f"Результирующая матрица доверия:\n{res_trust_matrix}")

x(0) = [  1   1   1   1   1   1   1   1 -37   1]
x(1) = [-5.8389 -4.3709 -0.5674 -3.7224  0.0886 -1.2574 -1.7744 -2.6765 -0.303
 -4.029 ]
x(2) = [-2.5922 -1.9905 -2.0026 -2.3674 -2.7046 -2.5931 -3.1355 -2.084  -3.0314
 -2.8545]
x(3) = [-2.4222 -2.3935 -2.5679 -2.5577 -2.3848 -2.4943 -2.5464 -2.5961 -2.4733
 -2.4684]
x(4) = [-2.4916 -2.5138 -2.5078 -2.4791 -2.4989 -2.493  -2.5014 -2.5071 -2.4832
 -2.4936]
x(5) = [-2.4999 -2.497  -2.4983 -2.4974 -2.497  -2.4971 -2.4937 -2.4962 -2.4966
 -2.4964]
x(6) = [-2.4969 -2.497  -2.4967 -2.497  -2.4975 -2.4973 -2.4974 -2.4969 -2.4972
 -2.4972]
x(7) = [-2.497  -2.497  -2.4971 -2.4971 -2.497  -2.4971 -2.497  -2.4971 -2.4971
 -2.497 ]
x(8) = [-2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971
 -2.4971]
x(9) = [-2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971
 -2.4971]
x(10) = [-2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971
 -2.4971]
x(11) = [-2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2

После взаимодействия агентов, вектор мнений сходится к значению:
X = [-2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971 -2.4971
 -2.4971]
Результирующая матрица доверия:
[[0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]
 [0.1113 0.0956 0.1393 0.1122 0.0708 0.0984 0.0911 0.1011 0.092  0.0882]]


In [155]:
if np.isclose(final_opinions[0], 0):
    print("Ничья")
elif final_opinions[0] > 0:
    print("Выиграл первый игрок")
else:
    print("Выиграл второй игрок") 

Выиграл второй игрок
